In [1]:
import pandas  as pd
import numpy as np

In [2]:
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

In [3]:
symbols = ['aapl', 'btc', 'eth', 'spx']

In [4]:
for symbol in symbols:
    data = pd.read_pickle('data/stocktwits/datasets_clean/' + symbol + '_twits_012018_072019_clean.pickle')
    
    data = data.replace({'sentiment_declared' : {'Bearish' : 0, 'Bullish' : 1}})
    
    data.index = pd.to_datetime(data.created_at)
    data = data.tz_convert('America/New_York')
    data = data[data.index > pd.to_datetime('2018-07-01 04:00:00+00:00').tz_convert('America/New_York')]
    
    data = data.dropna()

    data['pos'] = (data.sentiment_declared == 1) + 0
    data['neg'] = (data.sentiment_declared == 0) + 0
    
    data = data[['pos', 'neg']].groupby(data.index.date).sum()
    
    price = pd.read_csv('data/prices/' + symbol + '_prices.csv', header=None, index_col=0)
    price.index = pd.to_datetime(price.index)
    data = data.join(price, how='inner')[['pos', 'neg']]
    data.to_csv('data/posneg/posneg_stw_' + symbol + '_dcl.csv')